# Route between multiple Runnables

This notebook covers how to do routing in the LangChain Expression Language

Right now, the easiest way to do it is to write a function that will take in the input of a previous step and return a **runnable**. Importantly, this should return a **runnable** and NOT actually execute.

Let's take a look at this with a simple example. We will create a simple example where we will first classify whether the user input is a question about LangChain, OpenAI, or other, and route to a corresponding prompt chain.

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

First, lets create a dummy chain that will return either 1 or 0, randomly

In [20]:
chain = PromptTemplate.from_template("""Given the user question below, classify it as either being about `LangChain`, `OpenAI`, or `Other`.

<question>
{question}
</question>

Classification:""") | ChatOpenAI() | StrOutputParser()

In [17]:
chain.invoke({"question": "how do I call openAI?"})

AIMessage(content='OpenAI', additional_kwargs={}, example=False)

Now, let's create three sub chains:

In [21]:
langchain_chain = PromptTemplate.from_template("""You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatOpenAI()
openai_chain = PromptTemplate.from_template("""You are an expert in openai. \
Always answer questions starting with "As Sam Altman told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatOpenAI()
general_chain = PromptTemplate.from_template("""Respond to the following question:

Question: {question}
Answer:""") | ChatOpenAI()

In [38]:
def route(info):
    inputs = {"question": lambda x: x["question"]}
    if info["topic"] == "OpenAI":
        return inputs | openai_chain

    elif info["topic"] == "LangChain":
        return inputs | langchain_chain
    else:
        return inputs | general_chain

In [40]:
full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | RunnableLambda(route)

In [35]:
full_chain.invoke({"question": "how do I use OpenAI?"})

AIMessage(content="As Sam Altman told me, to use OpenAI, you can start by visiting the OpenAI website and exploring the available tools and resources. OpenAI offers a range of products that you can utilize, such as the GPT-3 language model or the Codex API. You can sign up for an account, read the documentation, and access the relevant APIs to integrate OpenAI's technologies into your applications. Additionally, you can join the OpenAI community to stay updated on the latest developments and connect with other users.", additional_kwargs={}, example=False)

In [36]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content="As Harrison Chase told me, to use LangChain, you will need to follow these steps:\n\n1. First, download and install the LangChain application on your device. It is available for both iOS and Android.\n\n2. Once installed, open the LangChain app and create an account. You will need to provide your email address and set a secure password.\n\n3. After creating your account, you will be prompted to select the languages you want to learn and the languages you already know. This will help tailor the learning experience to your specific needs.\n\n4. Once the initial setup is complete, you can start using LangChain to learn languages. The app offers various features such as interactive lessons, vocabulary exercises, and language exchange opportunities with native speakers.\n\n5. The app also provides personalized recommendations based on your learning progress and areas that need improvement. It tracks your performance and adjusts the difficulty level accordingly.\n\n6. Addi

In [41]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='The sum of 2 plus 2 is 4.', additional_kwargs={}, example=False)